In [2]:
import os
import shutil

In [3]:
base_image_path = 'brisc2025/segmentation_task/train/images'
nameImage = 'Sorted_Images'
folders_set = set()

bip = os.listdir(base_image_path)

for b in bip:
    remove_jpg =  b.replace(".jpg", "")
    split_b = remove_jpg.split('_')
    folder = '_'.join(split_b[3:6])
    folders_set.add(folder)
    
    if not os.path.isdir(nameImage):
        os.mkdir(nameImage)
        print(f"Folder '{nameImage}' created.") 

for folder in folders_set:
        sub = os.path.join(nameImage, folder)
        os.makedirs(sub, exist_ok=True)
        print(f"Folder '{sub}' created successfully.") 

In [4]:
base_mask_path = 'brisc2025/segmentation_task/train/masks'
nameMask = 'Sorted_Masks'
folders_set = set()

bip = os.listdir(base_mask_path)

for b in bip:
    remove_jpg =  b.replace(".jpg", "")
    split_b = remove_jpg.split('_')
    folder = '_'.join(split_b[3:6])
    folders_set.add(folder)
    print(folders_set)
    if not os.path.isdir(nameMask):
        os.mkdir(nameMask)
        print(f"Folder '{nameMask}' created.") 

for folder in folders_set:
        sub = os.path.join(nameMask, folder)
        os.makedirs(sub, exist_ok=True)
        print(f"Folder '{sub}' created successfully.") 

In [5]:
source_d = 'brisc2025/segmentation_task/train/masks' #change for basedir
dest_d = 'Sorted_Masks/pi_sa_t1' #change accordingly for subdir

for images in os.listdir(source_d):
    s_path = os.path.join(source_d, images)
    d_path = os.path.join(dest_d, images)

    if os.path.isfile(s_path):
        if images.__contains__('pi_sa_t1'): #chnage accordingly for subdir
            shutil.move(s_path, d_path)
            print(f"Moved '{images}' to '{d_path}'")

In [6]:
sort_images = 'Sorted_Images'
sort_masks = 'Sorted_Masks'


for dirpath, dirname, files in os.walk(sort_images):
    for f1 in files:
        sort_image_path = os.path.join(dirpath, f1) 
        
for dirpath, dirname, files in os.walk(sort_masks):
    for f2 in files:
        sort_mask_path = os.path.join(dirpath, f2)
        split_smp = sort_mask_path.split('_')
        code_numberM = split_smp[5]
    
    for f in files:
        if f.__contains__(code_numberM):
                print(sort_mask_path)
                print(code_numberM)


Sorted_Masks/gl_co_t1/brisc2025_train_00479_gl_co_t1.jpg
00479
Sorted_Masks/gl_ax_t1/brisc2025_train_00153_gl_ax_t1.jpg
00153
Sorted_Masks/gl_sa_t1/brisc2025_train_00953_gl_sa_t1.jpg
00953
Sorted_Masks/pi_sa_t1/brisc2025_train_04842_pi_sa_t1.jpg
04842
Sorted_Masks/me_sa_t1/brisc2025_train_02459_me_sa_t1.jpg
02459
Sorted_Masks/me_ax_t1/brisc2025_train_01148_me_ax_t1.jpg
01148
Sorted_Masks/pi_ax_t1/brisc2025_train_03957_pi_ax_t1.jpg
03957
Sorted_Masks/pi_co_t1/brisc2025_train_04391_pi_co_t1.jpg
04391
Sorted_Masks/me_co_t1/brisc2025_train_01891_me_co_t1.jpg
01891


In [7]:
numkeys = []

for dirpath, dirname, files in os.walk(sort_images):
    for f in files:
        jpg = f.replace(".jpg", "")
        split = jpg.split('_')
        numkey = split[2:6]
        joined_numkey = '_'.join(numkey)
        numkeys.append(joined_numkey) 

In [8]:
#d_(x) <- dierectory 
#dn_(x) <- directory name
#f_(x) <- files in direcotry 

batch_paths = []

for (d_m, dn_ms, f_m), (d_i, dn_is, f_i) in zip(os.walk(sort_masks), os.walk(sort_images)):
        for f1, f2 in zip(f_i, f_m):
               full_path_i = os.path.join(d_i, f1)
               full_path_m = os.path.join(d_m, f2)
               
               for n in numkeys:
                       if full_path_i.__contains__(n) and full_path_m.__contains__(n):
                               batch_paths.append((full_path_i, full_path_m))

for batch in batch_paths:
    print(batch)

('Sorted_Images/gl_co_t1/brisc2025_train_00597_gl_co_t1.jpg', 'Sorted_Masks/gl_co_t1/brisc2025_train_00597_gl_co_t1.jpg')
('Sorted_Images/gl_co_t1/brisc2025_train_00568_gl_co_t1.jpg', 'Sorted_Masks/gl_co_t1/brisc2025_train_00568_gl_co_t1.jpg')
('Sorted_Images/gl_co_t1/brisc2025_train_00814_gl_co_t1.jpg', 'Sorted_Masks/gl_co_t1/brisc2025_train_00814_gl_co_t1.jpg')
('Sorted_Images/gl_co_t1/brisc2025_train_00590_gl_co_t1.jpg', 'Sorted_Masks/gl_co_t1/brisc2025_train_00590_gl_co_t1.jpg')
('Sorted_Images/gl_co_t1/brisc2025_train_00396_gl_co_t1.jpg', 'Sorted_Masks/gl_co_t1/brisc2025_train_00396_gl_co_t1.jpg')
('Sorted_Images/gl_co_t1/brisc2025_train_00813_gl_co_t1.jpg', 'Sorted_Masks/gl_co_t1/brisc2025_train_00813_gl_co_t1.jpg')
('Sorted_Images/gl_co_t1/brisc2025_train_00418_gl_co_t1.jpg', 'Sorted_Masks/gl_co_t1/brisc2025_train_00418_gl_co_t1.jpg')
('Sorted_Images/gl_co_t1/brisc2025_train_00652_gl_co_t1.jpg', 'Sorted_Masks/gl_co_t1/brisc2025_train_00652_gl_co_t1.jpg')
('Sorted_Images/gl_co_t1

In [9]:
import torch
from torch.utils.data import Dataset
from PIL import Image

In [ ]:
PIL_batch = []

for  img_path, mask_path in batch_paths:
    
    image = Image.open(img_path)
    mask = Image.open(mask_path)
    PIL_batch.append((image, mask))

In [51]:
#after appending to PIL_batch list 
for p in PIL_batch:
    print(p)

(<PIL.JpegImagePlugin.JpegImageFile image mode=L size=512x512 at 0x11DEC86A0>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=512x512 at 0x11DEBF460>)
(<PIL.JpegImagePlugin.JpegImageFile image mode=L size=512x512 at 0x11D40AB00>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=512x512 at 0x11DEBF640>)
(<PIL.JpegImagePlugin.JpegImageFile image mode=L size=512x512 at 0x11DEBFC40>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=512x512 at 0x11DEBFB80>)
(<PIL.JpegImagePlugin.JpegImageFile image mode=L size=512x512 at 0x11DEC86D0>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=512x512 at 0x11DEC8520>)
(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x11DEC8FD0>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=512x512 at 0x11DEC84F0>)
(<PIL.JpegImagePlugin.JpegImageFile image mode=L size=512x512 at 0x11DEC9180>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=512x512 at 0x11DEC87C0>)
(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=

In [53]:
#check that images still line up
for batch in PIL_batch[38]: #just prints one tuple of images at that location 
    batch.show()